<a href="https://colab.research.google.com/github/nateraw/huggingface-hub-examples/blob/main/huggingface_hub_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install huggingface_hub
! git config --global credential.helper store

# Using the Hugging Face Hub Client Library

[![gh stars](https://img.shields.io/github/stars/huggingface/huggingface_hub?style=social)](https://github.com/huggingface/huggingface_hub)

Hey there! 👋

The following is a walkthrough of the `huggingface_hub` library. We'll go over how to create and manage repos, upload/download files, and how you can enable pushing/pulling any machine learning model to and from the Hugging Face Hub.

## Authentication

To begin, we'll want to make sure we're authenticated with the Hugging Face Hub.

> If you don't already have an account, you'll have to [sign up here](https://hf.co/join)!

This handy `notebook_login` function below surfaces up a nice little widget for us to log in using a token from [the tokens page](https://huggingface.co/settings/tokens). 

If you don't have one already, add one! ***Just make sure you specify that it has write access***, since we'll be pushing to the hub.
<!-- 
![add_token.gif](https://huggingface.co/datasets/nateraw/hf-hub-walkthrough-assets/resolve/main/add_token.gif) -->

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


We can use `huggingface_hub.whoami` to see our authenticated user's information. Here, we'll use it to grab our username so we can use it programmatically throughout this notebook.

In [ ]:
from huggingface_hub import whoami

user_info = whoami()
username = user_info['name']

print(f"You're logged in as: {username}. Your profile can be found at https://hf.co/{username}")

You're logged in as: nateraw. Your profile can be found at https://hf.co/nateraw


## Managing Repositories on the Hub

#### Create a Repo

Let's begin by creating a model repo using the `create_repo` function. We'll provide `exist_ok=True` to make sure we don't run into any errors if the repo already exists under our profile.

In [ ]:
from huggingface_hub import create_repo, delete_repo

repo_id = f"{username}/deleteme-test-repo"
repo_url = create_repo(repo_id, exist_ok=True)

print(f"Nice! We just created a repo at: {repo_url}")

Nice! We just created a repo at: https://huggingface.co/nateraw/deleteme-test-repo


#### Delete a Repo

We created the repo above just so we could see how to delete a repo. Let's try that now by using the `delete_repo` function.

⚠️ **Be careful with this function when using it in your own projects - it's final!** 

In [ ]:
delete_repo(repo_id)

print(f"RIP our beautiful repo. We should be seeing a 404 now at: {repo_url}")

RIP our beautiful repo. We should be seeing a 404 now at: https://huggingface.co/nateraw/deleteme-test-repo


💨 Poof! Just like that it's gone.

## Interacting with Repos

Let's create another repo. This time, let's see how we can interact with it. We'll name it something cool now since we don't plan on deleting this one right away.

In [ ]:
repo_id = f"{username}/my-cool-repo"

repo_url = create_repo(repo_id, exist_ok=True)
print(f"Made a shiny new model repo at: {repo_url}")

Made a shiny new model repo at: https://huggingface.co/nateraw/my-cool-repo


#### Upload a file to your repo

Let's see how we can upload a file to the repo. To do that, we'll need something to upload. So, let's create a dummy text file called `test_file.txt`.

In [ ]:
filepath = 'test_file.txt'
with open(filepath, 'w') as f:
    f.write("Hello, world!")

Now lets upload that file to our repo using the `upload_file` function. This function takes in a few required keyword arguments:

- path_or_fileobj: Path to a file on the local machine or binary data stream / fileobj / buffer.
- path_in_repo: Relative filepath in the repo. Ex: `"checkpoints/lfec34a/weights.bin"`
- repo_id: The repository to which the file will be uploaded. Ex: `"username/my-cool-repo"`

In [ ]:
from huggingface_hub import upload_file

file_url = upload_file(
    path_or_fileobj=filepath,
    path_in_repo=filepath,
    repo_id=repo_id,
)
print(f"Done! Check out the uploaded file here: {file_url}")

Done! Check out the uploaded file here: https://huggingface.co/nateraw/my-cool-repo/blob/main/test_file.txt


#### Downloading a file from your repo

Now that we've got something up there, let's download it using the `hf_hub_download` function.

Here, we just have to specify the repo and the filepath in the repo that we wish to download. In our case, its `"test_file.txt"`. 

In [ ]:
from huggingface_hub import hf_hub_download

cache_filepath = hf_hub_download(repo_id, filepath)
print(cache_filepath)

Downloading:   0%|          | 0.00/13.0 [00:00<?, ?B/s]

/root/.cache/huggingface/hub/c4ec643a7f6799023b0f36fbe8adaf26fad5875d935042e40a55c7d4b8abe4e8.31f3cc0f08a75796b7017d7c5e8611156388c341fd140724a79da30e67c9a6aa


📝 This file is actually a version controlled cache file. If we go to download it again, it'll just load from cache if it's the latest revision in your repo.

Let's open that file up and read its contents to make sure it matches what we wrote above.

In [ ]:
with open(cache_filepath, 'r') as f:
    text = f.read()
print(text)

Hello, world!


And it does! Sweet.

## Deleting a file in your repo

Let's delete `"test_file.txt"` using the `delete_file` function so we can clean up our repo for the next example.

In [ ]:
from huggingface_hub import delete_file

delete_file(filepath, repo_id)

🗑 Awesome, the file is gone now! Let's move on to a more exciting example...

## Pushing Machine Learning Models to the Hugging Face Hub

So we've seen how we can manage repos and files on the Hub. Now, let's put these tools to work and upload an actual model. 🚀

## 🚧 This part is a WIP, just started! 🚧

> I'm trying to figure out what's the best way to go here. Is it even to show a ML model? Is PyTorch going to make people think it only works for PyTorch?
> 
> One thought I have is to make a simple custom object that pushes a dict or something? Like a custom pretrained config class, almost. Idk.

In [ ]:
import torch
from torch.nn import Sequential, Linear

input_dim, output_dim = 784, 10
model = Sequential(Linear(input_dim, output_dim))

tensor([[ 0.4113, -0.0800,  0.2622, -0.1242, -0.5018,  0.1229, -0.0310, -0.2269,
         -0.2883, -0.2686]], grad_fn=<AddmmBackward0>)

In [ ]:
x = torch.ones(1, input_dim)
output = model(x)
print(f"Shape of output: {output.shape}")

Shape of output: torch.Size([1, 10])
